# Interactive Playwright Exploration

This notebook lets you explore ProjectSight interactively with Playwright.

**Tips:**
- Run cells one at a time
- The browser will open in a visible window (headless=False)
- Use `page.pause()` to open Playwright Inspector for picking locators
- Session is saved automatically after login for reuse

In [19]:
# Setup - run this first
from playwright.async_api import async_playwright
from pathlib import Path
import asyncio
from dotenv import load_dotenv
import os

load_dotenv(r'/home/pdcur/samsung-project/.env')

USERNAME = os.getenv('PROJECTSIGHT_USERNAME_2')
PASSWORD = os.getenv('PROJECTSIGHT_PASSWORD_2')

print(f"Using username: {USERNAME}")

Using username: pcuriel@mxi.pro


In [20]:
# Session storage path
storage_path = Path.home() / ".projectsight_session.json"

# Target URL
TARGET_URL = "https://prod.projectsightapp.trimble.com/web/app/Project?listid=-4045&orgid=4540f425-f7b5-4ad8-837d-c270d5d09490&projid=3"

# Store playwright instance globally
pw = None
browser = None
context = None
page = None

print(f"Session file: {storage_path}")
print(f"Session exists: {storage_path.exists()}")

Session file: /home/pdcur/.projectsight_session.json
Session exists: True


In [21]:
# Launch browser with saved session (if available)
pw = await async_playwright().start()
browser = await pw.chromium.launch(headless=False)

# Load saved session if it exists
context_options = {"viewport": {"width": 1920, "height": 1080}}
if storage_path.exists():
    context_options["storage_state"] = str(storage_path)
    print("Loading saved session...")
else:
    print("No saved session, will need to login")

context = await browser.new_context(**context_options)
page = await context.new_page()
print("Browser launched!")

Loading saved session...
Browser launched!


In [22]:
# Navigate to target page
await page.goto(TARGET_URL, wait_until="domcontentloaded")
await page.wait_for_timeout(3000)  # Give it a moment to redirect if needed

print(f"Current URL: {page.url}")

# Check if we need to login
needs_login = "id.trimble.com" in page.url or "sign_in" in page.url

if needs_login:
    print(">>> Login required - run the login cells below")
    
    try:
        await page.get_by_role("button", name="Reject All").click(timeout=3000)
        print("Dismissed cookie modal")
    except:
        print("No cookie modal appeared")
        
    # Login Step 1: Enter username
    await page.fill('#username-field', USERNAME)
    # The Next button may be disabled until field is blurred, so we simulate a Tab key press
    await page.keyboard.press('Tab') 
    await page.get_by_role("button", name="Next").click()
    print("Username entered, clicked Next")

    # Wait for password field
    await page.wait_for_selector('input[name="password"]', timeout=5000)
    print("Password field appeared")
    
    # Login Step 2: Enter password and submit
    await page.fill('input[name="password"]', PASSWORD)
    # The Sign in button may be disabled until field is blurred, so we simulate a Tab key press
    await page.keyboard.press('Tab') 
    await page.get_by_role("button", name="Sign in").click()
    print("Password entered, logging in...")

    # Wait for redirect to ProjectSight
    await page.wait_for_url('**projectsight**', timeout=10000)
    print(f"Logged in! URL: {page.url}")

    # Save session for next time
    await context.storage_state(path=str(storage_path))
    print(f"Session saved to {storage_path}")
        
else:
    print(">>> Already authenticated! Skip to the exploration cells")

Current URL: https://prod.projectsightapp.trimble.com/web/app/Project?listid=-4045&orgid=4540f425-f7b5-4ad8-837d-c270d5d09490&projid=3
>>> Already authenticated! Skip to the exploration cells


## Exploration

Now you're logged in! Use these cells to explore.

In [24]:
await page.pause()

In [ ]:
# Get the iframe and then the filter input                                                           
frame = page.locator("iframe[name=\"fraMenuContent\"]").content_frame                                
spec_section_input = frame.locator("#ucSearchPanel_ctl38_txtCSICodeLookupInput") 

# 1. Click to open the dropdown 
await spec_section_input.click(timeout=2000)

# 2. Type to filter the list                                                                         
await frame.locator("#ucSearchPanel_ctl38_txtCSICodeLookupInput").click(timeout=2000)

try:
    # Click on the Metal division expand button if not expanded yet
    await frame.locator("#ucSearchPanel_ctl38_CSICodePopupTreeContainer_73003").get_by_title("Expand Row").click(timeout=2000)
except:
    print("Row already expanded")
await frame.get_by_role("gridcell", name="05 12 00 - Structural Steel").click(timeout=2000) 
# await frame.locator("#ucSearchPanel_ctl38_txtCSICodeLookupInput").type("05 12 00", timeout=5000)
#  

page.wait_for_timeout(5000) # Wait to see the selection
await frame.locator("#imgSwitchToListView").click(timeout=2000)  # Switch to list view for easier parsing
print("Switched to list view")

Row already expanded


In [63]:
# Find the scrollable table container                                                                
scroll_container = frame.locator(".ui-iggrid-scrolldiv-y, .ui-iggrid-virtualization-container, [class*='scroll']").first
print(scroll_container)

<Locator frame=<Frame name= url='https://prod.projectsightapp.trimble.com/web/app/Project?listid=-4045&orgid=4540f425-f7b5-4ad8-837d-c270d5d09490&projid=3'> selector='iframe[name="fraMenuContent"] >> internal:control=enter-frame >> .ui-iggrid-scrolldiv-y, .ui-iggrid-virtualization-container, [class*=\'scroll\'] >> nth=0'>


In [61]:
processed = 0                                                                                        
seen_texts = set()  # Track unique rows to avoid duplicates 

while True: 
    rows = frame.locator("tbody tr")
    count = await rows.count()
    
    # Proces new rows:
    for i in range(processed, count):
        row = rows.nth(i)
        text = await row.inner_text()

        # Skip if already seen (virtual scrolling can re-render)                                     
        if text in seen_texts:                                                                       
            continue     
                                                                                    
        seen_texts.add(text)
        print(f"Row {i+1}: {text.replace(chr(10), ' | ')}")  # Replace newlines for better readability
        
    processed = count
    
    # Scroll to the last row to trigger loading more rows
    await rows.last.scroll_into_view_if_needed()
    await frame.wait_for_timeout(2000)  # Wait for new rows to load
    
    # Check if more rows were loaded
    new_count = await rows.count()
    if new_count == count:
        print("No more new rows to load.")
        break
    
print("Finished processing all rows.")
print(f"Total unique rows processed: {len(seen_texts)}")

Row 1:  |                  |                     My views |                  |                  | 					 |                      |                      |                  |             
Row 2: 	 | Number | 	 | Revision	 | Spec section	 | Spec sub section	 | Subject	 | Status	 | Due date	 | Open assignments	 |  
Row 3: 	 | Number | 	 | Revision	 | Spec section	 | Spec sub section	 | Subject	 | Status	 | Due date	 | Open assignments	
Row 4: 	0317	0	05 12 00 - Structural Steel Framing		TF1-SMT-SH-0841_061 - 05 12 00 - 0 - TRS - Structural Steel - Trestle A, B, E, F, & M Anchor Bolt - SD	 Approved with Comments	4/25/2023		 |  | 	0356	0	05 12 00 - Structural Steel Framing		TF1-SMT-SH-0855_062 - 05 12 00 - 0 - GCS - Structural Steel - Shell Steel - GL 1-11 - SD	 Approved with Comments	5/1/2023		 |  | 	0357	0	05 12 00 - Structural Steel Framing		TF1-SMT-SH-0856_063 - 05 12 00 - 0 - GCS - Structural Steel - Shell Steel - GL 11-20 - SD	 Approved with Comments	5/1/2023		 |  | 	0373	0	05 12 00 - St

Task exception was never retrieved
future: <Task finished name='Task-157' coro=<Channel.send() done, defined at /home/pdcur/samsung-project/.venv/lib/python3.12/site-packages/playwright/_impl/_connection.py:61> exception=Exception('Channel.send: Connection closed while reading from the driver')>
Traceback (most recent call last):
  File "/home/pdcur/samsung-project/.venv/lib/python3.12/site-packages/playwright/_impl/_connection.py", line 69, in send
    return await self._connection.wrap_api_call(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/pdcur/samsung-project/.venv/lib/python3.12/site-packages/playwright/_impl/_connection.py", line 559, in wrap_api_call
    raise rewrite_error(error, f"{parsed_st['apiName']}: {error}") from None
Exception: Channel.send: Connection closed while reading from the driver
Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed\nCall log:\n  - waiting for

TimeoutError: Locator.scroll_into_view_if_needed: Timeout 29985ms exceeded.
Call log:
  - attempting scroll into view action
    2 × waiting for element to be stable
      - element is not visible
    - retrying scroll into view action
    - waiting 20ms
    2 × waiting for element to be stable
      - element is not visible
    - retrying scroll into view action
      - waiting 100ms
    58 × waiting for element to be stable
       - element is not visible
     - retrying scroll into view action
       - waiting 500ms


In [ ]:
# Take a screenshot
await page.screenshot(path="screenshot.png")
print("Screenshot saved to screenshot.png")

## Cleanup

In [ ]:
# Cleanup - run when done
await browser.close()
await pw.stop()
print("Browser closed")

In [ ]:
# Optional: Delete saved session (to force fresh login next time)
if storage_path.exists():
    storage_path.unlink()
    print("Session deleted")
else:
    print("No session to delete")